<a href="https://colab.research.google.com/github/DenisKai7/id_scanner/blob/model-train/90_generate_labels_scanner_ktp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -qq
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-ind
!pip install pytesseract tqdm

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-ind
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 537 kB of archives.
After this operation, 1,138 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-ind all 1:4.00~git30-7274cfa-1.1 [537 kB]
Fetched 537 kB in 1s (1,058 kB/s)
Selecting previously unselected package tesseract-ocr-ind.
(Reading database ... 126284 files and directorie

In [3]:
import os
import pandas as pd
import pytesseract
import cv2
from glob import glob
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

DATASET_PATH = '/content/drive/MyDrive/ocr_ktp/train'
FOLDER = '90'

img_files = glob(os.path.join(DATASET_PATH, FOLDER, '*.jpg'))
records = []
print(f"Proses folder {FOLDER} ({len(img_files)} file)")

for img_path in tqdm(img_files):
    try:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        try:
            text = pytesseract.image_to_string(img, lang='ind')
            if not text.strip():
                text = pytesseract.image_to_string(img, lang='eng')
        except:
            text = pytesseract.image_to_string(img, lang='eng')
        label_text = text.replace('\n', ' ').strip()
        records.append({'filename': img_path, 'labeltext': label_text})
    except Exception as e:
        print(f"Error di {img_path}: {e}")
        records.append({'filename': img_path, 'labeltext': ''})

# Simpan hasil per folder
df = pd.DataFrame(records)
csv_out = f"{DATASET_PATH}/labels_{FOLDER}.csv"
df.to_csv(csv_out, index=False)
print(f"Selesai folder {FOLDER}, file label: {csv_out}")

Mounted at /content/drive
Proses folder 90 (4825 file)


100%|██████████| 4825/4825 [1:36:17<00:00,  1.20s/it]

Selesai folder 90, file label: /content/drive/MyDrive/ocr_ktp/train/labels_90.csv
